<a href="https://colab.research.google.com/github/aruntakhur/LLMs/blob/main/Fine_Tune_CoT_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Fine-Tune T5 with Chain-of-Thought (CoT) Reasoning
This Colab notebook fine-tunes `flan-t5-small` on CoT-style data, following the paper [Large Language Models Are Reasoning Teachers (2022)](https://arxiv.org/abs/2212.10071).

In [ ]:

# ✅ Install required libraries
!pip install transformers datasets peft accelerate --quiet


In [ ]:

# ✅ Import libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model


In [ ]:

# ✅ Load tokenizer and model (Flan-T5)
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:

# ✅ Apply LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, peft_config)


In [ ]:
from pathlib import Path
# ✅ Sample CoT-style training data (You can expand this)
import json
Path("data").mkdir(exist_ok=True)
sample_data = [
    {
        "input": "Q: If a train travels at 60 km/h for 2 hours, how far does it go?\nA: Let's think step by step.",
        "target": "The train travels at 60 km/h. In 2 hours, it will travel 60 × 2 = 120 km. Therefore, the answer is 120 km."
    },
    {
        "input": "Q: What is 15 + 27?\nA: Let's think step by step.",
        "target": "15 + 27 = 42. Therefore, the answer is 42."
    }
]
with open("data/finetune_cot_train.json", "w") as f:
    json.dump(sample_data, f)


In [ ]:
import json
from datasets import Dataset

# Load the JSON file manually
with open("data/finetune_cot_train.json", "r") as f:
    data = json.load(f)

# Convert to Hugging Face Dataset
train_ds = Dataset.from_list(data)


In [ ]:

# ✅ Tokenize the dataset
def tokenize(batch):
    input_encodings = tokenizer(batch["input"], truncation=True, padding="max_length", max_length=128)
    target_encodings = tokenizer(batch["target"], truncation=True, padding="max_length", max_length=128)
    input_encodings["labels"] = target_encodings["input_ids"]
    return input_encodings

train_ds = train_ds.map(tokenize, batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:

# ✅ Training configuration
training_args = TrainingArguments(
    output_dir="./cot-t5-small",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=5,
    save_steps=20,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer
)

trainer.train()


In [ ]:

# ✅ Inference (test on new question)
input_text = "Q: If you have 10 candies and eat 4, how many are left?\nA: Let's think step by step."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))
